In [1]:
!pip install langchain-google-genai langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.5 MB/s eta 0:00:00


In [2]:
!pip install gdown
!gdown --id 1tiBCCYPjeIN92TkO8Vt8vrpSKLmGb-6Y -O UNBench-all.zip
!unzip UNBench-all.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: UNBench-all/meetings_adopted/2547/S_PV.9699-EN_statements.json  
  inflating: UNBench-all/meetings_adopted/2547/S_PV.9699-EN.json  
  inflating: UNBench-all/meetings_adopted/864/S_PV.3571-EN_statements.json  
  inflating: UNBench-all/meetings_adopted/864/S_PV.3571-EN.json  
  inflating: UNBench-all/meetings_adopted/2123/S_PV-7790-EN.json  
  inflating: UNBench-all/meetings_adopted/2123/S_PV-7790-EN_statements.json  
  inflating: UNBench-all/meetings_adopted/2311/S_PV.8701-EN_statements.json  
  inflating: UNBench-all/meetings_adopted/2311/S_PV.8701-EN.json  
  inflating: UNBench-all/meetings_adopted/1594/S_PV.5732-EN.json  
  inflating: UNBench-all/meetings_adopted/1594/S_PV.5732-EN_statements.json  
  inflating: UNBench-all/meetings_adopted/2329/S_2020_460-EN.json  
  inflating: UNBench-all/meetings_adopted/1558/S_PV.5610-EN_statements.json  
  inflating: UNBench-all/meetings_adopted/1558/S_PV.5610-EN.json  
  inflating: UNBench-all/meetings_ado

In [3]:
import json

with open('/content/UNBench-all/task3_test.json', 'r') as f:
    data = json.load(f)

test_texts = data['drafts']
test_labels = data['labels']

In [5]:
!pip install langchain_google_genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=userdata.get('VERTEX_API_KEY'),
    temperature=0,
    vertexai=True
)

In [6]:
from tqdm import tqdm
import pandas as pd
import random
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
from langchain_core.messages import SystemMessage, HumanMessage

def classify_texts(texts):
    results = []
    for text in tqdm(texts):
        user_prompt = f"""
        The provided document is a United Nations Security Council's draft resolution. Predict whether the draft resolution will be adopted or not. Answer with 'yes' (1) or 'no' (0) without any explanation.

        Text: "{text}"
        Answer:
        """
        messages = [
            SystemMessage(content="You are a helpful assistant."),
            HumanMessage(content=user_prompt)
        ]
        response = llm.invoke(messages)
        result = response.content.strip().lower()

        if result.startswith("yes") or result == "1":
            results.append(1)
        elif result.startswith("no") or result == "0":
            results.append(0)
        else:
            results.append(random.choice([0, 1]))
    return results

pred = classify_texts(test_texts)

100%|██████████| 989/989 [1:12:44<00:00,  4.41s/it]


In [7]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, average_precision_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, precision_recall_curve, auc

def calculate_metrics(pred, labels):
    # swap 0 and 1
    pred = [1 - x for x in pred]
    labels = [1 - x for x in labels]
    acc = accuracy_score(labels, pred)
    try:
        roc_auc = roc_auc_score(labels, pred)
    except ValueError:
        roc_auc = 0
    balanced_acc = balanced_accuracy_score(labels, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    # pr_auc = average_precision_score(labels, pred)
    precision, recall, _ = precision_recall_curve(labels, pred)
    pr_auc = auc(recall, precision)
    mcc = matthews_corrcoef(labels, pred)
    g_mean = geometric_mean_score(labels, pred)
    tn, fp, fn, tp = confusion_matrix(labels, pred).ravel()
    specificity = tn / (tn + fp)

    print(f'Accuracy: {acc}')
    print(f'AUC: {roc_auc}')
    print(f'Balanced Accuracy: {balanced_acc}')
    print(f'Precision: {prec}')
    print(f'Recall: {rec}')
    print(f'F1: {f1}')
    print(f'PR AUC: {pr_auc}')
    print(f'MCC: {mcc}')
    print(f'G-Mean: {g_mean}')
    print(f'Specificity: {specificity}')

    print('Accuracy AUC Balanced_Acc Precision Recall F1 PR_AUC MCC G-Mean Specificity')
    print(f'{acc:.4f} {roc_auc:.4f} {balanced_acc:.4f} {prec:.4f} {rec:.4f} {f1:.4f} {pr_auc:.4f} {mcc:.4f} {g_mean:.4f} {specificity:.4f}')

In [8]:
calculate_metrics(pred, test_labels)

Accuracy: 0.9696663296258847
AUC: 0.9305757135945815
Balanced Accuracy: 0.9305757135945816
Precision: 0.6619718309859155
Recall: 0.8867924528301887
F1: 0.7580645161290323
PR AUC: 0.7774155089454636
MCC: 0.7512852629407566
G-Mean: 0.9295451494192747
Specificity: 0.9743589743589743
Accuracy AUC Balanced_Acc Precision Recall F1 PR_AUC MCC G-Mean Specificity
0.9697 0.9306 0.9306 0.6620 0.8868 0.7581 0.7774 0.7513 0.9295 0.9744
